<a href="https://colab.research.google.com/github/adrianlapaz3/Procesamiento_Lenguaje_Natural/blob/main/PNL_TP2_Adrian_Lapaz.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --upgrade --force-reinstall --no-cache-dir numpy
!pip install --upgrade --force-reinstall --no-cache-dir tensorflow
!pip install --upgrade --force-reinstall --no-cache-dir keras
!pip install --upgrade --force-reinstall --no-cache-dir gensim

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.8/16.8 MB 147.1 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.19.0 requires numpy<2.2.0,>=1.26.0, but you have numpy 2.2.6 which is incompatible.
gensim 4.3.3 requires numpy<2.0,>=1.18.5, but you have numpy 2.2.6 which is incompatible.
numba 0.60.0 requires numpy<2.1,>=1.22, but you have numpy 2.2.6 which is incompatible.
tf-keras 2.18.0 requires tensorflow<2.19,>=2.18, but you have tensorflow 2.19.0 which is incompatible.
tensorflow-decision-forests 1.11.0 requires tensorflow==2.18.0, but you have tensorflow 2.19.0 which is incompatible.
tsfresh 0.21.0 req

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 30.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.7/135.7 kB 164.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 174.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.8/16.8 MB 130.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 201.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 416.8/416.8 kB 197.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.5/66.5 kB 154.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 243.2/243.2 kB 191.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.5/87.5 kB 165.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 205.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.8/45.8 kB 146.9 MB/s eta 0:00:00
  Attempting uninstall: namex
    Found existi

In [ ]:
!pip install gensim

  Using cached scipy-1.13.1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (60 kB)
Using cached scipy-1.13.1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (38.6 MB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tsfresh 0.21.0 requires scipy>=1.14.0; python_version >= "3.10", but you have scipy 1.13.1 which is incompatible.


In [ ]:
import re
import multiprocessing
import logging
import nltk
from nltk.corpus import gutenberg, stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import numpy as np
import pandas as pd
from gensim.models import Word2Vec
from gensim.models.callbacks import CallbackAny2Vec
from sklearn.manifold import TSNE
import sklearn
from packaging.version import parse as parse_version
import plotly.express as px
import plotly.graph_objects as go


# --- Enable Gensim Logging ---
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

# --- NLTK Resource Download Function ---
def download_nltk_resource(resource_path, resource_name):
    try:
        nltk.data.find(resource_path)
        logging.info(f"NLTK resource '{resource_name}' ({resource_path}) already downloaded.")
    except LookupError:
        logging.info(f"NLTK resource '{resource_name}' ({resource_path}) not found. Downloading...")
        nltk.download(resource_name, quiet=False)
        logging.info(f"NLTK resource '{resource_name}' downloaded.")

# Download necessary NLTK resources
download_nltk_resource('corpora/gutenberg', 'gutenberg')
download_nltk_resource('tokenizers/punkt', 'punkt')
download_nltk_resource('corpora/stopwords', 'stopwords')
download_nltk_resource('corpora/wordnet', 'wordnet')
download_nltk_resource('corpora/omw-1.4', 'omw-1.4')
print("NLTK resource checks complete.")
print("Setup and imports complete.")



[nltk_data] Downloading package gutenberg to /root/nltk_data...
[nltk_data]   Unzipping corpora/gutenberg.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


NLTK resource checks complete.
Setup and imports complete.


In [ ]:
import nltk
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [ ]:
# ==============================================================================
# "Hamlet" Corpus
# ==============================================================================
corpus_id = 'shakespeare-hamlet.txt'
try:
    raw_hamlet_text = gutenberg.raw(corpus_id)
    logging.info(f"Successfully loaded raw text for '{corpus_id}'. Length: {len(raw_hamlet_text)} characters.")
except LookupError:
    logging.error(f"Corpus '{corpus_id}' not found in NLTK Gutenberg. Please ensure it's downloaded.")
    raw_hamlet_text = ""

# Isolate actual dialogue. This is iterative and may need further refinement.
# Words to specifically track during preprocessing for debugging
debug_words_to_trace = {"love", "revenge", "madness", "skull"}

processed_lines_for_tokenization = []
if raw_hamlet_text:
    start_marker = "[The Tragedie of Hamlet by William Shakespeare"
    end_marker = "*** END OF THE PROJECT GUTENBERG EBOOK HAMLET ***"

    try:
        start_index = raw_hamlet_text.lower().find(start_marker.lower())
        actual_text_start = 0
        if start_index != -1:
            end_of_header_line_index = raw_hamlet_text.find('\n', start_index + len(start_marker))
            if end_of_header_line_index != -1:
                 actual_text_start = end_of_header_line_index + 1
            else:
                 actual_text_start = start_index + len(start_marker)
            raw_hamlet_text = raw_hamlet_text[actual_text_start:]
            logging.info(f"Attempted to remove Gutenberg header. New start of text preview: {raw_hamlet_text[:200]}")

        end_index = raw_hamlet_text.lower().find(end_marker.lower())
        if end_index != -1:
            raw_hamlet_text = raw_hamlet_text[:end_index]
            logging.info("Attempted to remove Gutenberg footer.")
    except Exception as e:
        logging.error(f"Error during header/footer removal: {e}")

    lines = raw_hamlet_text.splitlines()

    act_scene_pattern = r"^\s*(ACTUS|ACT|SCENE|SCOENA)\s+[IVXLCDM]+\.?\s*$"
    speaker_pattern_advanced = r"^([A-Z][A-Z\s\.]{0,25}\.)\s*"
    stage_direction_bracket_pattern = r"\[.*?\]"
    stage_direction_paren_pattern = r"\(.*?\)"

    for line_number, line_content_original in enumerate(lines):
        line = line_content_original.strip()
        if not line:
            continue

        # Debug: Check if line contains any debug words BEFORE any filtering
        for dw in debug_words_to_trace:
            if dw in line.lower():
                logging.debug(f"Line {line_number+1} RAW: Contains '{dw}'. Line: '{line_content_original}'")

        if re.fullmatch(act_scene_pattern, line, re.IGNORECASE):
            logging.debug(f"Line {line_number+1}: Filtered Act/Scene: {line_content_original}")
            continue

        # Remove bracketed and parenthesized stage directions first
        line_no_brackets = re.sub(stage_direction_bracket_pattern, "", line)
        line_no_parens = re.sub(stage_direction_paren_pattern, "", line_no_brackets)
        line = line_no_parens.strip()

        # Attempt to remove speaker tags
        cleaned_after_speaker_removal = re.sub(speaker_pattern_advanced, "", line)
        if len(cleaned_after_speaker_removal) < len(line):
            logging.debug(f"Line {line_number+1}: Speaker pattern matched. Original segment: '{line}', After speaker removal: '{cleaned_after_speaker_removal}'")
            line = cleaned_after_speaker_removal.strip()

        if not line:
            logging.debug(f"Line {line_number+1}: Became empty after cleaning stage directions/speaker. Original: '{line_content_original}'")
            continue

        # Filter lines that are likely just remaining stage directions or very short non-dialogue
        if (line.isupper() and len(line.split()) < 5) or len(line.split()) < 2: # Require at least 2 words for context
            logging.debug(f"Line {line_number+1}: Filtered short/uppercase line: '{line}' from original: '{line_content_original}'")
            continue

        processed_lines_for_tokenization.append(line)

logging.info(f"Number of lines after structural filtering: {len(processed_lines_for_tokenization)}")

lemmatizer = WordNetLemmatizer()
english_stopwords = set(stopwords.words('english'))

def tokenize_lemmatize_remove_stopwords(text_line, line_idx_for_debug="Unknown"):
    # Tokenize first
    tokens_raw = word_tokenize(text_line.lower())

    # Debug: Check if any of the debug_words are present after initial tokenization
    for dw in debug_words_to_trace:
        if dw in tokens_raw:
            logging.debug(f"Line {line_idx_for_debug} DEBUG_TRACE (Post-Tokenize): Word '{dw}' found. Tokens: {tokens_raw}")

    # Keep only alphabetic tokens
    alpha_tokens = [word for word in tokens_raw if word.isalpha()]
    for dw in debug_words_to_trace:
        if dw not in alpha_tokens and dw in tokens_raw:
             logging.debug(f"Line {line_idx_for_debug} DEBUG_TRACE (Post-IsAlpha): Word '{dw}' REMOVED by isalpha. Original tokens: {tokens_raw}, Alpha: {alpha_tokens}")
        elif dw in alpha_tokens:
             logging.debug(f"Line {line_idx_for_debug} DEBUG_TRACE (Post-IsAlpha): Word '{dw}' KEPT after isalpha. Alpha: {alpha_tokens}")


    # Lemmatize
    lemmatized_tokens = [lemmatizer.lemmatize(word) for word in alpha_tokens]
    for dw in debug_words_to_trace:
        lem_dw = lemmatizer.lemmatize(dw)
        if lem_dw not in lemmatized_tokens and dw in alpha_tokens:
            if dw != lem_dw :
                 logging.debug(f"Line {line_idx_for_debug} DEBUG_TRACE (Post-Lemma): Word '{dw}' changed to '{lem_dw}'. Lemmatized: {lemmatized_tokens}")

        elif lem_dw in lemmatized_tokens:
             logging.debug(f"Line {line_idx_for_debug} DEBUG_TRACE (Post-Lemma): Word '{dw}' (as '{lem_dw}') KEPT. Lemmatized: {lemmatized_tokens}")


    # Remove stopwords and short words (length 1)
    final_processed_tokens = [word for word in lemmatized_tokens if word not in english_stopwords and len(word) > 1]
    for dw in debug_words_to_trace:
        lem_dw = lemmatizer.lemmatize(dw)
        if lem_dw not in final_processed_tokens and lem_dw in lemmatized_tokens:
            reason = "stopword" if lem_dw in english_stopwords else "length<=1" if len(lem_dw) <=1 else "unknown"
            logging.debug(f"Line {line_idx_for_debug} DEBUG_TRACE (Post-Stopword/Len): Word '{dw}' (as '{lem_dw}') REMOVED. Reason: {reason}. Lemmatized before filter: {lemmatized_tokens}")
        elif lem_dw in final_processed_tokens:
            logging.debug(f"Line {line_idx_for_debug} DEBUG_TRACE (Post-Stopword/Len): Word '{dw}' (as '{lem_dw}') KEPT. Final: {final_processed_tokens}")

    return final_processed_tokens

sentences_for_word2vec = []
if processed_lines_for_tokenization:
    for idx, line_text in enumerate(processed_lines_for_tokenization):
        processed_tokens = tokenize_lemmatize_remove_stopwords(line_text, line_idx_for_debug=str(idx+1)) # Pass index
        if processed_tokens and len(processed_tokens) > 1:
            sentences_for_word2vec.append(processed_tokens)

if sentences_for_word2vec:
    print(f"Preprocessing complete. Number of effectively processed sentences for Word2Vec: {len(sentences_for_word2vec)}")
    print("\nExample of first 5 processed sentences (post-lemmatization & stopword removal):")
    for i in range(min(5, len(sentences_for_word2vec))):
        print(sentences_for_word2vec[i])
else:
    print("No sentences available after full preprocessing. Review filtering, corpus content, and Gensim INFO logs.")


Preprocessing complete. Number of effectively processed sentences for Word2Vec: 3735

Example of first 5 processed sentences (post-lemmatization & stopword removal):
['actus', 'primus', 'scoena', 'prima']
['enter', 'barnardo', 'francisco', 'two', 'centinels']
['fran', 'nay', 'answer', 'stand', 'vnfold']
['bar', 'long', 'liue', 'king']
['fran', 'barnardo']


El preprocesamiento del texto de Hamlet encontró 3735 oraciones efectivas que es una cantidad aceptable para un modelo Word2Vec. Los Tokens incluyen:
* Indicadores estructurales de la obra (ej., `['actus', 'primus', 'scoena', 'prima']`) para que el modelo aprenda el contexto de éstos.
* Direcciones de escena y nombres de personajes (ej., `['enter', 'barnardo', 'francisco']`, `['fran', 'nay', 'answer']`) que son utiles para analizar interacciones de personajes.
* Formas antiguas de escribir palabras (ej., `vnfold`, `liue`) por ende, el modelo aprenderá embeddings para estas formas específicas.
* Buena base para Word2Vec ya que se obtuvo una lematización ("raíces de palabras") y eliminación de palabras vacias (ej. "the") mejoran la calidad de los tokens para el entrenamiento.


In [ ]:
# ==============================================================================
# Train Gensim Word2Vec Model
# ==============================================================================

class LossCallback(CallbackAny2Vec):
    def __init__(self):
        self.epoch = 0
        self.cumulative_loss_at_previous_epoch = 0.0

    def on_epoch_end(self, model):
        current_cumulative_loss = model.get_latest_training_loss()
        loss_this_epoch = current_cumulative_loss - self.cumulative_loss_at_previous_epoch
        logging.info(f"Epoch {self.epoch}: Cumulative loss = {current_cumulative_loss:.4f}, Loss this epoch = {loss_this_epoch:.4f}")
        print(f"Epoch {self.epoch}: Cumulative loss = {current_cumulative_loss:.4f}, Loss this epoch = {loss_this_epoch:.4f}")
        self.cumulative_loss_at_previous_epoch = current_cumulative_loss
        self.epoch += 1

w2v_model_hamlet = None
wv_hamlet = None

MIN_SENTENCES_FOR_TRAINING = 50
if sentences_for_word2vec and len(sentences_for_word2vec) >= MIN_SENTENCES_FOR_TRAINING:
    print(f"Starting Word2Vec model training with {len(sentences_for_word2vec)} sentences...")

    w2v_model_hamlet = Word2Vec(
        sentences=sentences_for_word2vec,
        vector_size=100,
        window=5,
        min_count=1,
        sg=1,
        hs=0,
        negative=10,
        workers=max(1, multiprocessing.cpu_count() - 1),
        epochs=100,
        compute_loss=True,
        callbacks=[LossCallback()],
        alpha=0.025,
        min_alpha=0.0001
    )

    wv_hamlet = w2v_model_hamlet.wv
    print(f"\nWord2Vec model training complete.")
    print(f"New Vocabulary size (with min_count=1): {len(wv_hamlet.index_to_key)}")
else:
    print(f"Skipping Word2Vec model training: Not enough processed sentences available (found {len(sentences_for_word2vec) if sentences_for_word2vec else 0}, need at least {MIN_SENTENCES_FOR_TRAINING}).")
    logging.warning(f"Word2Vec training skipped. Number of processed sentences: {len(sentences_for_word2vec) if sentences_for_word2vec else 0}")


Starting Word2Vec model training with 3735 sentences...
Epoch 0: Cumulative loss = 274219.2500, Loss this epoch = 274219.2500
Epoch 1: Cumulative loss = 531738.9375, Loss this epoch = 257519.6875
Epoch 2: Cumulative loss = 725770.8125, Loss this epoch = 194031.8750
Epoch 3: Cumulative loss = 883290.1250, Loss this epoch = 157519.3125
Epoch 4: Cumulative loss = 1023993.8750, Loss this epoch = 140703.7500
Epoch 5: Cumulative loss = 1157890.2500, Loss this epoch = 133896.3750
Epoch 6: Cumulative loss = 1287612.1250, Loss this epoch = 129721.8750
Epoch 7: Cumulative loss = 1414280.8750, Loss this epoch = 126668.7500
Epoch 8: Cumulative loss = 1540271.0000, Loss this epoch = 125990.1250
Epoch 9: Cumulative loss = 1663991.5000, Loss this epoch = 123720.5000
Epoch 10: Cumulative loss = 1787845.2500, Loss this epoch = 123853.7500
Epoch 11: Cumulative loss = 1909797.2500, Loss this epoch = 121952.0000
Epoch 12: Cumulative loss = 2030220.8750, Loss this epoch = 120423.6250
Epoch 13: Cumulative l

El entrenamiento del modelo Word2Vec con 3735 oraciones procesadas de "Hamlet" a lo largo de 100 épocas demuestra un aprendizaje efectivo, evidenciado por la progresiva disminución del "Loss this epoch" (de 274k a 98k). Aunque la estabilización del loss hacia el final sugiere una convergencia adecuada para este corpus específico y de tamaño limitado, el uso de min_count=1 resultó en un vocabulario de 4145 palabras. Esto indica que los embeddings generados serán altamente especializados en el lenguaje y contexto de "Hamlet", si bien la representación vectorial de términos muy poco frecuentes podría ser menos robusta debido a su escaso contexto en los datos de entrenamiento.

In [ ]:
# ==============================================================================
# Test Terms of Interest and Explain Similarities
# ==============================================================================

if wv_hamlet:
    words_of_interest = ["hamlet", "king", "queen", "ghost", "ophelia", "death",
                         "play", "revenge", "love", "madness", "skull", "denmark",
                         "polonius", "laertes", "horatio", "gertrude", "claudius", "fortinbras",
                         "poison", "sword", "yorick", "doubt", "fate", "honour"]

    for term in words_of_interest:
        if term in wv_hamlet:
            try:
                similar_words = wv_hamlet.most_similar(term, topn=5)
                print(f"\nMost similar to '{term}':")
                for word, score in similar_words:
                    print(f"  {word}: {score:.4f}")
            except KeyError:
                print(f"\n'{term}' was in vocab but KeyError during most_similar (unexpected).")
        else:
            print(f"\n'{term}' not found in vocabulary (likely due to frequency or preprocessing).")

    pairs_to_compare = [
        ("hamlet","ophelia"), ("king","queen"), ("death","ghost"),
        ("play","revenge"), ("hamlet", "madness"), ("king", "claudius"),
        ("ophelia", "love"), ("laertes", "revenge"), ("poison", "death")
    ]
    print("\nPair similarities:")
    for w1, w2 in pairs_to_compare:
        if w1 in wv_hamlet and w2 in wv_hamlet:
            try:
                similarity = wv_hamlet.similarity(w1, w2)
                print(f"  {w1} - {w2}: {similarity:.4f}")
            except KeyError:
                 print(f"  Could not calculate similarity for {w1}-{w2} (unexpected KeyError).")
        else:
            print(f"  Cannot compare '{w1}' and '{w2}': one or both not in vocabulary.")
else:
    print("Word2Vec model (wv_hamlet) not available for similarity testing.")


Most similar to 'hamlet':
  vnbrac: 0.5818
  doublet: 0.5772
  dar: 0.5572
  combate: 0.5508
  postscript: 0.5363

Most similar to 'king':
  discouery: 0.5048
  authority: 0.4761
  earnest: 0.4677
  secricie: 0.4654
  rosincrance: 0.4642

Most similar to 'queen':
  willow: 0.7905
  aslant: 0.7713
  inobled: 0.7246
  nightly: 0.7182
  vertuous: 0.6852

Most similar to 'ghost':
  gent: 0.6654
  adulterate: 0.6547
  seate: 0.6359
  ambass: 0.6319
  befitted: 0.5916

Most similar to 'ophelia':
  beautifed: 0.7816
  idoll: 0.7765
  sprung: 0.7581
  interre: 0.7351
  hugger: 0.7309

Most similar to 'death':
  forged: 0.7482
  warrantie: 0.7476
  hush: 0.7395
  obscure: 0.7158
  guiltlesse: 0.7157

Most similar to 'play':
  villanous: 0.6786
  hart: 0.6780
  considered: 0.6716
  vngalled: 0.6537
  indifferently: 0.6229

'revenge' not found in vocabulary (likely due to frequency or preprocessing).

'love' not found in vocabulary (likely due to frequency or preprocessing).

'madness' not found

**Resultados de similitud de "Hamlet"**

La evaluación de los embeddings de "Hamlet" mediante `most_similar` y la similitud coseno entre pares específicos revela la capacidad y las limitaciones del modelo Word2Vec entrenado con un corpus literario único.

**1. Cobertura del vocabulario y su impacto semántico:**
Se constata una limitación crítica debida a la ausencia de términos temáticamente centrales como `love`, `madness`, `revenge`, `skull` y `poison` en el vocabulario del modelo. Esta exclusión, atribuible a la frecuencia de los términos post-preprocesamiento o a los umbrales de `min_count`, impide una representación semántica completa de dimensiones narrativas fundamentales de "Hamlet". Los embeddings resultantes, por tanto, operan sobre un subconjunto léxico que no abarca la totalidad conceptual de la obra.

**2. Captura de asociaciones contextuales específicas (`most_similar`):**
El modelo ha demostrado ser efectivo en la captura de asociaciones semánticas altamente específicas y contextualmente relevantes para "Hamlet" cuando los términos están presentes en su vocabulario. Esto se fundamenta en la hipótesis distribucional, donde la proximidad vectorial refleja la similitud de los contextos de co-ocurrencia.
* Para `ophelia`, se observan asociaciones como `beautifed`, `idoll`, `orizons` y `nimph` (similitudes >0.73), términos directamente vinculados a su caracterización y a las descripciones textuales.
* La relación de `queen` con `willow` y `aslant` (similitudes >0.77) es particularmente notable, pues codifica elementos descriptivos específicos de la narrativa de la muerte de `Ophelia` narrada por `Gertrude`.
* Para `ghost`, la similitud con `adulterate` (0.6547) refleja una conexión temática central.
* Términos como `denmark` y `fortinbras` se asocian con un léxico geopolítico coherente (`polake`, `warres`, `prison` para `denmark`; `compact`, `slay`, `inheritance` para `fortinbras`), con similitudes a menudo >0.79.
* El modelo también aprende representaciones para grafías arcaicas (ej., `discouery` para `king`) y formas léxicas específicas del preprocesamiento.
* No obstante, se identifican asociaciones menos intuitivas (ej., para `hamlet`: `vnbrac`, `doublet`, `dar`, con similitudes de 0.53-0.58), que podrían interpretarse como ruido estadístico, la influencia de co-ocurrencias esporádicas en un corpus reducido, o la captura de contextos muy locales y específicos (vestimenta, apariencia) en lugar de relaciones semánticas más abstractas. Las puntuaciones de similitud en estos casos tienden a ser moderadas.

**3. Relaciones semánticas cuantitativas  (similitud entre pares):**
La similitud coseno entre pares seleccionados ofrece una cuantificación de su cercanía en el espacio vectorial aprendido:
* La relación `king` - `claudius` (0.4284) indica una asociación semántica moderada, reflejando su identidad parcialmente superpuesta dentro de la narrativa.
* Pares como `hamlet` - `ophelia` (0.2943) y `king` - `queen` (0.2157) exhiben similitudes más débiles. Esto puede sugerir que, si bien narrativamente conectados, sus perfiles distribucionales (los contextos lingüísticos en los que aparecen) no son lo suficientemente intercambiables en el corpus como para generar una alta similitud coseno. Alternativamente, la complejidad de sus relaciones podría no traducirse en una simple proximidad vectorial basada en co-ocurrencia.
* La similitud `death` - `ghost` (0.2938) es también moderada-baja, posiblemente indicando que, aunque conceptualmente ligados, sus contextos de uso en "Hamlet" son suficientemente distintos.
* La imposibilidad de calcular similitudes para pares que incluyen términos ausentes del vocabulario (`play`/`revenge`, `hamlet`/`madness`, etc.) reitera el impacto de la cobertura léxica en la capacidad analítica del modelo.


In [ ]:
# ==============================================================================
# Test Analogy Tests
# ==============================================================================

if wv_hamlet:
    a1, b1, c1 = "king", "man", "woman"
    expected1 = "queen"
    print(f"\nAnalogy Test 1: {a1} - {b1} + {c1} (expecting ~{expected1})")
    if all(word in wv_hamlet for word in [a1, b1, c1]):
        try:
            results_direct = wv_hamlet.most_similar(positive=[a1, c1], negative=[b1], topn=10)
            print(f"  Directly via most_similar(positive=['{a1}', '{c1}'], negative=['{b1}']):")
            found_direct = any(res[0] == expected1 for res in results_direct)
            for word, score in results_direct: print(f"    {word}: {score:.4f}")
            if found_direct: print(f"    -> Expected '{expected1}' was found!")

            vec_a1 = wv_hamlet[a1]; vec_b1 = wv_hamlet[b1]; vec_c1 = wv_hamlet[c1]
            fabricated_vec1 = vec_a1 - vec_b1 + vec_c1
            results_manual = wv_hamlet.most_similar(positive=[fabricated_vec1], topn=10)
            print("\n  Manually (vector(king) - vector(man) + vector(woman)) then finding similar:")
            found_manual = any(res[0] == expected1 for res in results_manual)
            for word, score in results_manual:
                 if word not in [a1,b1,c1]: print(f"    {word}: {score:.4f}")
            if found_manual: print(f"    -> Expected '{expected1}' was found!")
            if not found_direct and not found_manual: print(f"    -> Expected '{expected1}' not prominent.")
        except KeyError as e:
            print(f"  Skipping Analogy Test 1 due to missing word: {e}")
        except Exception as e:
            print(f"  An error occurred during Analogy Test 1: {e}")
    else:
        print(f"  Skipping Analogy Test 1: One or more input words ({a1},{b1},{c1}) not in vocabulary.")

    a2, b2, c2 = "laertes", "polonius", "ghost"
    expected2 = "hamlet"
    print(f"\nAnalogy Test 2: {a2} (son1) - {b2} (father1) + {c2} (father2) (expecting ~{expected2} (son2))")
    if all(word in wv_hamlet for word in [a2, b2, c2]):
        try:
            results_direct2 = wv_hamlet.most_similar(positive=[a2, c2], negative=[b2], topn=5)
            print(f"  Directly via most_similar(positive=['{a2}', '{c2}'], negative=['{b2}']):")
            found_direct2 = any(res[0] == expected2 for res in results_direct2)
            for word, score in results_direct2: print(f"    {word}: {score:.4f}")
            if found_direct2: print(f"    -> Expected '{expected2}' was found!")
            else: print(f"    -> Expected '{expected2}' not prominent.")
        except KeyError as e:
            print(f"  Skipping Analogy Test 2 due to missing word: {e}")
        except Exception as e:
            print(f"  An error occurred during Analogy Test 2: {e}")
    else:
        print(f"  Skipping Analogy Test 2: One or more input words ({a2},{b2},{c2}) not in vocabulary.")
else:
    print("Word2Vec model (wv_hamlet) not available for analogy testing.")



Analogy Test 1: king - man + woman (expecting ~queen)
  Directly via most_similar(positive=['king', 'woman'], negative=['man']):
    hide: 0.4740
    ignorant: 0.4675
    fox: 0.4555
    pernicious: 0.4426
    smiling: 0.4266
    confound: 0.4266
    palpable: 0.4243
    arraigne: 0.4156
    trade: 0.4147
    amaze: 0.4099

  Manually (vector(king) - vector(man) + vector(woman)) then finding similar:
    hide: 0.3585
    ignorant: 0.3571
    fox: 0.3380
    arraigne: 0.3333
    palpable: 0.3268
    kinde: 0.3244
    tribute: 0.3150
    bring: 0.3105
    -> Expected 'queen' not prominent.

Analogy Test 2: laertes (son1) - polonius (father1) + ghost (father2) (expecting ~hamlet (son2))
  Directly via most_similar(positive=['laertes', 'ghost'], negative=['polonius']):
    lesse: 0.5069
    occurrent: 0.4814
    anticipation: 0.4404
    pittied: 0.4388
    exclaim: 0.4105
    -> Expected 'hamlet' not prominent.


**Tests de Analogías**

Los tests de analogías (king - man + woman ≈ queen; laertes - polonius + ghost ≈ hamlet) no identificaron los términos canónicos. Los vectores resultantes de las operaciones aritméticas presentaron bajas similitudes coseno (0.3-0.5) con los términos más cercanos del vocabulario, como hide o lesse.

Este resultado se atribuye principalmente a:

Especificidad del Corpus y Ausencia de Subestructuras Lineales Generalizables: Los embeddings, entrenados exclusivamente en "Hamlet" (3700 oraciones), reflejan un contexto semántico altamente especializado. Este corpus limitado y de dominio único impide la formación robusta de las subestructuras lineales (vector offsets consistentes) necesarias para que el modelo capture las relaciones semánticas abstractas (género, realeza, roles familiares) que estas analogías evalúan. Las asociaciones narrativas específicas de la obra dominan sobre estas relaciones lingüísticas generales.

Pobre Alineación de Vectores de Analogía: Las bajas puntuaciones de similitud indican que los vectores "fabricados" por las operaciones de analogía no se alinean fuertemente con ningún vector de palabra existente en el espacio aprendido de "Hamlet". Esto sugiere que las relaciones semánticas específicas que se intentan aislar mediante la aritmética vectorial no están representadas como vectores distintivos y bien definidos en este modelo.

En conclusión, si bien los embeddings pueden capturar similitudes contextuales locales dentro de "Hamlet", el espacio semántico aprendido carece de la generalización y estructura relacional necesarias para resolver estas tareas de analogía, una limitación esperable dada la naturaleza del corpus de entrenamiento.

In [ ]:
# ==============================================================================
# Graph the Resulting Embeddings (t-SNE)
# ==============================================================================

RANDOM_STATE = 0

def reduce_dimensions_class_style(keyed_vectors_obj, num_target_dimensions=2, random_seed=0):

    if not keyed_vectors_obj or not hasattr(keyed_vectors_obj, 'vectors') or \
       not hasattr(keyed_vectors_obj, 'index_to_key') or len(keyed_vectors_obj.index_to_key) == 0:
        print("Error: KeyedVectors object is None, empty, or not a valid Gensim KeyedVectors instance.")
        return None, None

    all_vectors = np.asarray(keyed_vectors_obj.vectors)
    all_labels = np.asarray(keyed_vectors_obj.index_to_key)

    default_perplexity = 30.0
    if len(all_vectors) <= default_perplexity or len(all_vectors) <= num_target_dimensions:
        print(
            f"Warning: Not enough samples ({len(all_vectors)}) for t-SNE with default perplexity ({default_perplexity}) "
            f"and/or target dimensions ({num_target_dimensions}). t-SNE might fail or produce poor results. "
            "Consider a smaller vocabulary or different t-SNE parameters if issues arise."
        )

    try:
        tsne_reducer = TSNE(n_components=num_target_dimensions, random_state=random_seed)
        transformed_vectors = tsne_reducer.fit_transform(all_vectors)
        return transformed_vectors, all_labels
    except Exception as e:
        print(f"Error during t-SNE transformation (n_components={num_target_dimensions}): {e}")
        return None, all_labels

# --- Perform Dimensionality Reductions ---
transformed_vecs_2d = None
all_vocab_labels_2d = None
transformed_vecs_3d = None
all_vocab_labels_3d = None


if wv_hamlet:
    print("Reducing dimensions for 2D t-SNE plot...")
    transformed_vecs_2d, all_vocab_labels_2d = reduce_dimensions_class_style(wv_hamlet, 2, RANDOM_STATE)
    if transformed_vecs_2d is not None:
        print(f"2D projection generated for {len(all_vocab_labels_2d)} words.")
    else:
        print("Failed to generate 2D projection.")

    print("\nReducing dimensions for 3D t-SNE plot...")

    transformed_vecs_3d, all_vocab_labels_3d = reduce_dimensions_class_style(wv_hamlet, 3, RANDOM_STATE)
    if transformed_vecs_3d is not None:
        print(f"3D projection generated for {len(all_vocab_labels_3d)} words.")
    else:
        print("Failed to generate 3D projection.")
else:
    print("Word2Vec model's KeyedVectors (wv_hamlet) not available. Skipping t-SNE.")

MAX_WORDS_TO_DISPLAY = 200

Reducing dimensions for 2D t-SNE plot...
2D projection generated for 4145 words.

Reducing dimensions for 3D t-SNE plot...
3D projection generated for 4145 words.


Las visualizaciones t-SNE (t-Distributed Stochastic Neighbor Embedding) constituyen una técnica no lineal de reducción de dimensionalidad usada para representar relaciones de similitud semántica entre palabras, a partir de sus embeddings vectoriales. Basadas en la hipótesis distribucional, según la cual las palabras que aparecen en contextos similares tienden a compartir significados, estas visualizaciones buscan preservar las relaciones de vecindad local del espacio original de alta dimensión. Los embeddings generados por modelos como Word2Vec capturan patrones de coocurrencia del corpus (en este caso, Hamlet) y los proyectan en un espacio vectorial. El algoritmo t-SNE, al reducir estas representaciones a 2D o 3D, facilita una inspección cualitativa de la semántica aprendida, donde la proximidad espacial entre puntos es un proxy de similitud contextual.

In [ ]:
import plotly.express as px

if transformed_vecs_2d is not None and all_vocab_labels_2d is not None:
    num_display = min(MAX_WORDS_TO_DISPLAY, len(all_vocab_labels_2d))

    print(f"\n--- Plotting 2D t-SNE Projection for top {num_display} words ---")
    try:
        fig_2d = px.scatter(
            x=transformed_vecs_2d[:num_display, 0],
            y=transformed_vecs_2d[:num_display, 1],
            text=all_vocab_labels_2d[:num_display],
            title=f"2D t-SNE Projection (Hamlet Embeddings, Top {num_display} Words)"
        )
        fig_2d.update_traces(textposition='top center', mode='markers+text', marker={'size': 5})
        fig_2d.update_layout(height=800, width=1000, xaxis_title="t-SNE Dimension 1", yaxis_title="t-SNE Dimension 2")
        fig_2d.show(renderer="colab")
    except Exception as e:
        print(f"Error displaying 2D Plotly figure: {e}")
else:
    print("Data for 2D plot (transformed_vecs_2d or labels) not available. Check Cell 1.")


--- Plotting 2D t-SNE Projection for top 200 words ---


**2D t-SNE**

En la proyección 2D se observa una organización general útil para captar agrupamientos temáticos, aunque con cierto solapamiento en regiones de alta densidad. Es destacable la agrupación densa en la parte superior derecha, donde coexisten "hamlet", "claudius", "ghost", "horatio", "king", "queen", "gertrude" y "polonius", lo que refleja la prominencia de estos personajes en contextos comunes. Sin embargo, la bidimensionalidad impone ciertas limitaciones: palabras clave como "himselfe", "ere", "grace" o "sword" aparecen más aisladas, sin indicar vínculos explícitos, aunque semánticamente relevantes. La ausencia de profundidad dificulta la observación de conexiones entre capas semánticas intermedias y palabras periféricas de baja frecuencia, algunas de las cuales podrían estar artificialmente alejadas del núcleo por limitaciones inherentes al modelo t-SNE en 2D.


In [ ]:
import plotly.express as px

if transformed_vecs_3d is not None and all_vocab_labels_3d is not None:
    # Ensure MAX_WORDS_TO_DISPLAY is not greater than available words
    num_display = min(MAX_WORDS_TO_DISPLAY, len(all_vocab_labels_3d))

    print(f"\n--- Plotting 3D t-SNE Projection for top {num_display} words ---")
    try:
        fig_3d = px.scatter_3d(
            x=transformed_vecs_3d[:num_display, 0],
            y=transformed_vecs_3d[:num_display, 1],
            z=transformed_vecs_3d[:num_display, 2],
            text=all_vocab_labels_3d[:num_display],
            title=f"3D t-SNE Projection (Hamlet Embeddings, Top {num_display} Words)"
        )
        fig_3d.update_traces(marker_size=3, mode='markers+text')
        fig_3d.update_layout(height=800, width=1000, scene=dict(aspectmode='data'))
        fig_3d.show(renderer="colab")
    except Exception as e:
        print(f"Error displaying 3D Plotly figure: {e}")
else:
    print("Data for 3D plot (transformed_vecs_3d or labels) not available. Check Cell 1.")


--- Plotting 3D t-SNE Projection for top 200 words ---


**3D t-SNE**

Las visualizaciones tridimensionales ofrecen una representación más rica y matizada del espacio semántico. En el gráfico 3D, los clusters observados en la vista 2D se reafirman pero se organizan de manera más coherente. Por ejemplo, el grupo "hamlet", "claudius", "ghost", "horatio", "king", "queen", "gertrude" aparece en una región más cohesiva, con una clara estructura esférica que indica fuerte cohesión semántica local. Además, se revelan subestructuras que no son evidentes en 2D: términos como "madness", "death", "revenge" y "father" parecen formar un subclúster temático que representa la trama psicológica de la obra. Palabras aisladas en la vista 2D (como "himselfe", "grace", "take", "earth") se integran de manera más natural en el espacio tridimensional, sugiriendo que su exclusión aparente era más un artefacto de la reducción bidimensional. En conjunto, las vistas 3D no solo confirman agrupamientos semánticos esperados, sino que revelan gradientes y transiciones contextuales que aportan una capa adicional de interpretabilidad.
